# Question : 회원들의 점수가 권장 에버리지 구간에 적합한가?

### Import

In [1]:
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 

plt.rcParams['font.family'] = 'AppleGothic'
warnings.filterwarnings('ignore')

### Load

In [2]:
file_path = './data'

df = pd.read_csv(file_path + '/billiards_processing.csv')
df.head()

,닉네임,점수,에버리지,날짜,종일,주간,야간,심야
0,도치,32,0.936,2023-04-01,0,1,0,0
1,큐사리,30,0.865,2023-04-01,0,1,0,0
2,레쓰비,30,0.797,2023-04-01,1,0,0,0
3,이장님,30,0.778,2023-04-01,0,1,0,0
4,서병관,30,0.723,2023-04-01,0,0,0,0


In [3]:
# 권장 에버리지 구간 정의
recommendations = {
    32: (0.950, 1.049),
    30: (0.860, 0.949),
    28: (0.780, 0.859),
    27: (0.710, 0.779),
    26: (0.650, 0.709),
    25: (0.600, 0.649),
    24: (0.550, 0.599),
    23: (0.500, 0.549),
    22: (0.450, 0.499),
    21: (0.400, 0.449),
    20: (0.380, 0.399),
    19: (0.360, 0.379),
    18: (0.340, 0.359),
    17: (0.320, 0.339),
    16: (0.300, 0.319)
}

In [19]:
grouped = df.groupby('닉네임').agg({'점수': 'first', '에버리지': 'mean'}).reset_index()
grouped

,닉네임,점수,에버리지
0,Ebs,25,0.560
1,가신,25,0.480
2,강드롱,22,0.428
3,건초,28,0.673
4,계란후라이,30,0.696
5,공심이,20,0.389
6,광따라지,20,0.354
7,김도형,27,0.645
8,김주영,21,0.406
9,눈사람,23,0.473


In [18]:
# 점수별 평균 에버리지 계산
score_avg_ever = grouped.groupby('점수')['에버리지'].mean().reset_index()

# 결과 출력
print(score_avg_ever)

    점수      에버리지
0   16  0.196000
1   17  0.296000
2   20  0.343857
3   21  0.376500
4   22  0.417500
5   23  0.430200
6   24  0.496167
7   25  0.551200
8   26  0.584667
9   27  0.649000
10  28  0.678200
11  30  0.771800
12  32  0.936000


In [11]:
# 핸디에 따른 권장 에버리지 구간 추가
def get_recommended_range(handicap):
    return recommendations.get(handicap, (None, None))

grouped['에버리지_최소'], grouped['에버리지_최대'] = zip(*grouped['점수'].apply(get_recommended_range))

# 권장 구간에 맞는지 여부 판단
grouped['적합성'] = grouped.apply(lambda row: row['에버리지_최소'] <= row['에버리지'] <= row['에버리지_최대'], axis=1)

# Check
grouped.head()

,닉네임,점수,에버리지,에버리지_최소,에버리지_최대,적합성
0,Ebs,25,0.560,0.60,0.649,False
1,가신,25,0.480,0.60,0.649,False
2,강드롱,22,0.428,0.45,0.499,False
3,건초,28,0.673,0.78,0.859,False
4,계란후라이,30,0.696,0.86,0.949,False


In [12]:
# 적합성 통계 계산 (적합한 회원 수와 적합하지 않은 회원 수)
match_count = grouped['적합성'].sum()
total_count = len(grouped)
not_match_count = total_count - match_count

# 결과 출력
print(f"권장 에버리지 구간에 적합한 회원 수: {match_count}")
print(f"권장 에버리지 구간에 적합하지 않은 회원 수: {not_match_count}")
print(f"적합 비율: {match_count / total_count:.2%}")

권장 에버리지 구간에 적합한 회원 수: 7
권장 에버리지 구간에 적합하지 않은 회원 수: 44
적합 비율: 13.73%


In [14]:
# 적합하지 않은 회원 리스트 출력
print("\n적합하지 않은 회원 데이터:")
grouped[grouped['적합성'] == False][['닉네임', '점수', '에버리지', '에버리지_최소', '에버리지_최대']]


적합하지 않은 회원 데이터:


,닉네임,점수,에버리지,에버리지_최소,에버리지_최대
0,Ebs,25,0.560,0.60,0.649
1,가신,25,0.480,0.60,0.649
2,강드롱,22,0.428,0.45,0.499
3,건초,28,0.673,0.78,0.859
4,계란후라이,30,0.696,0.86,0.949
6,광따라지,20,0.354,0.38,0.399
7,김도형,27,0.645,0.71,0.779
9,눈사람,23,0.473,0.50,0.549
10,달무리,21,0.362,0.40,0.449
11,달보그레,26,0.572,0.65,0.709


In [16]:
# 적합하지 않은 회원 중 구간을 벗어난 회원 분리

# 더 높은 경우: 에버리지가 권장 최대값을 초과한 경우
higher_than_range = grouped[grouped['에버리지'] > grouped['에버리지_최대']]

# 더 낮은 경우: 에버리지가 권장 최소값보다 작은 경우
lower_than_range = grouped[grouped['에버리지'] < grouped['에버리지_최소']]

# 더 높은 경우 출력
print("권장 구간을 벗어나 더 높은 경우:")
higher_than_range[['닉네임', '점수', '에버리지', '에버리지_최소', '에버리지_최대']]

# 더 낮은 경우 출력
print("\n권장 구간을 벗어나 더 낮은 경우:")
lower_than_range[['닉네임', '점수', '에버리지', '에버리지_최소', '에버리지_최대']]

권장 구간을 벗어나 더 높은 경우:

권장 구간을 벗어나 더 낮은 경우:


,닉네임,점수,에버리지,에버리지_최소,에버리지_최대
0,Ebs,25,0.560,0.60,0.649
1,가신,25,0.480,0.60,0.649
2,강드롱,22,0.428,0.45,0.499
3,건초,28,0.673,0.78,0.859
4,계란후라이,30,0.696,0.86,0.949
6,광따라지,20,0.354,0.38,0.399
7,김도형,27,0.645,0.71,0.779
9,눈사람,23,0.473,0.50,0.549
10,달무리,21,0.362,0.40,0.449
11,달보그레,26,0.572,0.65,0.709
